# Explore Results

In [133]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [134]:
from pathlib import Path
import pandas as pd
import os
import joblib
import pickle
from src.sklearn_utils import predict_fn, plot_one_vs_rest_success_rates, plot_confusion_matrix, compute_cross_entropy_loss
from sklearn.metrics import multilabel_confusion_matrix
import numpy as np

In [135]:
artifacts_dir = Path(os.path.abspath('')).parent / 'artifacts'
model_path = artifacts_dir / 'gradient_boosting-20220704-153024.joblib'
model = joblib.load(model_path)

In [136]:
model.cv_results_

{'mean_fit_time': array([ 21.69707355,  64.43889155, 132.12126284]),
 'std_fit_time': array([ 0.50096234,  1.39458765, 49.39470151]),
 'mean_score_time': array([0.06359849, 0.0473938 , 0.04681373]),
 'std_score_time': array([0.01985951, 0.00117783, 0.00224644]),
 'param_clf__base_estimator__max_depth': masked_array(data=[2, 4, 6],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'clf__base_estimator__max_depth': 2},
  {'clf__base_estimator__max_depth': 4},
  {'clf__base_estimator__max_depth': 6}],
 'split0_test_score': array([0.07306397, 0.07205387, 0.07474747]),
 'split1_test_score': array([0.05938567, 0.05699659, 0.05836177]),
 'split2_test_score': array([0.06622517, 0.06390728, 0.06523179]),
 'split3_test_score': array([0.05785953, 0.05785953, 0.05719064]),
 'split4_test_score': array([0.0620339 , 0.06372881, 0.06237288]),
 'mean_test_score': array([0.06371365, 0.06290922, 0.06358091]),
 'std_test_score': array([0.00546919, 0

In [137]:
mlb = pickle.load(open(artifacts_dir / 'gradient_boosting-20220704-153024-binarizer.pkl', 'rb'))

In [138]:
df = pd.read_csv(artifacts_dir / 'gradient_boosting-20220704-153024-test.csv')
# convert strings in column to list. e.g. "['A', 'B', 'C']" -> ['A', 'B', 'C'], "['Hello', 'World']" -> ['Hello', 'World']
df['sectors_list'] = df['sectors_list'].apply(lambda x: eval(x))
df["sectors_list"] = df["sectors_list"].apply(
    lambda x: [tag for tag in x if tag in mlb.classes_.tolist()]
)

In [139]:
y_true, y_pred, hl = predict_fn(df, mlb, model)

XGBoostError: [18:24:53] /home/stefan/PycharmProjects/multi-label-classification-climate/xgboost/src/c_api/../data/../common/device_helpers.cuh:428: Memory allocation error on worker 0: std::bad_alloc: cudaErrorUnknown: unknown error
- Free memory: 3966369792
- Requested memory: 1572

Stack trace:
  [bt] (0) /home/stefan/PycharmProjects/multi-label-classification-climate/venv/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x80) [0x7fcc9d0bdc60]
  [bt] (1) /home/stefan/PycharmProjects/multi-label-classification-climate/venv/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(dh::detail::ThrowOOMError(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, unsigned long)+0x434) [0x7fcc9d3bec14]
  [bt] (2) /home/stefan/PycharmProjects/multi-label-classification-climate/venv/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(+0x210e50) [0x7fcc9d05ee50]
  [bt] (3) /home/stefan/PycharmProjects/multi-label-c

In [140]:
naive_path = artifacts_dir / 'naive-stratified.joblib'
naive_model = joblib.load(naive_path)

In [141]:
y_pred_naive = naive_model.predict(df)

In [142]:
plot_one_vs_rest_success_rates(y_true, y_pred, y_pred_naive, classes=mlb.classes_)

# Confusion Matrices

In [143]:
mlbcm=multilabel_confusion_matrix(y_true, y_pred)

In [144]:
# TODO: Fix these confusion matrices. They are not correct.
# loop through multiple confusion matrices and plot them.
for ix, matrix in enumerate(mlbcm):
    class_label = mlb.classes_[ix]
    fig=plot_confusion_matrix(matrix, class_label, normalize=True)
    fig.show()

# Sort by loss

In this multilabel case, there are a number of ways of sorting by loss for error analysis. The simplest way is to plug the model's outputted probabilities and ground truth into a proper scoring rule (e.g. cross entropy loss) to compare the distance between the predicted distribution and the true distribution for each example. Models with a high difference of worthy of additional investigation. The non-linearity of the scoring rule should highlight instances where probabilities are particularly at odds with at least one label.

However, we can also sort by the worst one-vs-rest probability scores if we want to focus on where the model goes wrong for particular labels.

In [72]:
probas=model.predict_proba(df)

XGBoostError: [17:57:11] /home/stefan/PycharmProjects/multi-label-classification-climate/xgboost/src/c_api/../data/../common/device_helpers.cuh:428: Memory allocation error on worker 0: std::bad_alloc: cudaErrorUnknown: unknown error
- Free memory: 3966369792
- Requested memory: 1572

Stack trace:
  [bt] (0) /home/stefan/PycharmProjects/multi-label-classification-climate/venv/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x80) [0x7fcc9d0bdc60]
  [bt] (1) /home/stefan/PycharmProjects/multi-label-classification-climate/venv/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(dh::detail::ThrowOOMError(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, unsigned long)+0x434) [0x7fcc9d3bec14]
  [bt] (2) /home/stefan/PycharmProjects/multi-label-classification-climate/venv/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(+0x210e50) [0x7fcc9d05ee50]
  [bt] (3) /home/stefan/PycharmProjects/multi-label-c

In [73]:
# Cross entropy is a "proper" scoring rule that measures how much a probabilistic distribution differs from the true distribution. We can sort by cross entropy loss to potentially find something fruitful. see https://stats.stackexchange.com/questions/312780/why-is-accuracy-not-the-best-measure-for-assessing-classification-models

In [87]:
instance_cross_entropy_losses = [compute_cross_entropy_loss(y_true[i], probas[i]) for i in range(len(y_true))]
df['instance_cross_entropy_losse'] = instance_cross_entropy_losses
pred_strings = mlb.inverse_transform(y_pred)
df['sectors_list_predicted'] = [list(t) for t in pred_strings]

In [ ]:
def debug_examples(ground_truth, prediction, description_text, full_text):
    """
    Debug examples by looking at ground truth and predicted labels and comparing it with text.

    Returns:
        None
    """
    print(f"Worst example ground truth: {ground_truth} \n\nWorst example ground truth: {prediction}\n\n Worst example description text: \n\n{description_text}\n\n Worst example full_text: \n\n{full_text}")